In [1]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
import datetime
import multiprocessing 
import xarray as xr
import fsspec
import dask

In [2]:
n_workers = multiprocessing.cpu_count()
cluster = LocalCluster(n_workers=n_workers, processes=False)
cluster

LocalCluster(fb4cbada, 'inproc://192.168.39.162/42096/1', workers=16, threads=16, memory=127.76 GiB)

In [3]:
client = Client(cluster)
client

Client Scheduler: inproc://192.168.39.162/42096/1 Dashboard: http://192.168.39.162:8787/status,Cluster Workers: 16 Cores: 16 Memory: 127.76 GiB


In [4]:
url = 's3://noaa-nwm-retro-v2-zarr-pds'

In [5]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)
ds

Wall time: 30.4 s


<xarray.Dataset>
Dimensions:         (feature_id: 2729077, time: 227904)
Coordinates:
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1993-01-01 ... 2018-12-31T23:00:00
Data variables:
    elevation       (time, feature_id) float32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    order           (time, feature_id) int32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBucket         (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    q_lateral       (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    streamflow      (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity        (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [2]:
%%time

start_date = "2010-01-01"
end_date = "2010-01-31"
comid = [6177558]
nwm = NWM(start_date=start_date, end_date=end_date, comids=comid)
comids = [6177558,6140826,6162981,9343363,8479268,724696,4594599,1722317,2677104,8077030,4710396,5892356]

Wall time: 999 µs


In [16]:
%%time
a_comids = []
for c in comids:
    try:
        test = ds["streamflow"].sel(feature_id=c).sel(time=slice("2010-01-01", "2010-01-01"))
        a_comids.append(c)
    except KeyError:
        print(f"COMID: {c} not found")
comids = a_comids
comids

Wall time: 79 ms


[6177558,
 6140826,
 6162981,
 9343363,
 8479268,
 724696,
 4594599,
 1722317,
 2677104,
 8077030,
 4710396,
 5892356]

In [17]:
%%time
nwm.end_date = nwm.end_date + datetime.timedelta(days=1)
# variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
variables = ["streamflow", "velocity"]
comids = comids[:2]
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_streamflow = ds[variables].sel(feature_id=comids).sel(time=slice(
        f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}",
        f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
    )).compute()

ds_streamflow

Wall time: 25.2 s


<xarray.Dataset>
Dimensions:     (feature_id: 2, time: 792)
Coordinates:
  * feature_id  (feature_id) int32 6177558 6140826
    latitude    (feature_id) float32 41.31 41.46
    longitude   (feature_id) float32 -72.53 -71.84
  * time        (time) datetime64[ns] 2010-01-01 ... 2010-02-02T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 0.51 0.35 0.51 ... 0.31 0.52 0.31
    velocity    (time, feature_id) float64 0.44 0.42 0.45 0.42 ... 0.4 0.45 0.4
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [34]:
%%time
timeseries = ds_streamflow.to_dataframe()
metadata = {}
j = 1
for c in comids:
    for v in variables:
        metadata[f"column_{j}_units"] = str(ds_streamflow.data_vars[v].attrs["units"])
        j += 1

output = {}
first = True
i = 1
for idx, catchment in timeseries.groupby("feature_id"):
    i_meta = True
    for date, row in catchment.iterrows():
        d = date[1].strftime('%Y-%m-%d %H')
        if first:
            output[d] = [r for r in row[variables]]
        else:
            for r in row[variables]:
                output[d].append(r)          
        if i_meta:
            for v in variables:
                metadata[f"column_{i}"] = f"{v}-{idx}"
                i += 1
            i_meta = False
    first = False

metadata

Wall time: 1.27 s


{'column_1_units': 'm3 s-1',
 'column_2_units': 'ms-1',
 'column_3_units': 'm3 s-1',
 'column_4_units': 'ms-1',
 'column_1': 'streamflow-6177558',
 'column_2': 'velocity-6177558',
 'column_3': 'streamflow-6140826',
 'column_4': 'velocity-6140826'}

{'2010-01-01 00': [0.5099999886006117,
  0.4399999901652336,
  0.3499999921768904,
  0.41999999061226845],
 '2010-01-01 01': [0.5099999886006117,
  0.4499999899417162,
  0.3499999921768904,
  0.41999999061226845],
 '2010-01-01 02': [0.5199999883770943,
  0.4499999899417162,
  0.3499999921768904,
  0.41999999061226845],
 '2010-01-01 03': [0.549999987706542,
  0.4599999897181988,
  0.3499999921768904,
  0.41999999061226845],
 '2010-01-01 04': [0.5799999870359898,
  0.46999998949468136,
  0.3399999924004078,
  0.41999999061226845],
 '2010-01-01 05': [0.6099999863654375,
  0.46999998949468136,
  0.3399999924004078,
  0.41999999061226845],
 '2010-01-01 06': [0.6099999863654375,
  0.46999998949468136,
  0.3399999924004078,
  0.41999999061226845],
 '2010-01-01 07': [0.5999999865889549,
  0.46999998949468136,
  0.3399999924004078,
  0.41999999061226845],
 '2010-01-01 08': [0.6099999863654375,
  0.46999998949468136,
  0.3399999924004078,
  0.41999999061226845],
 '2010-01-01 09': [0.599999986588

In [10]:
client.close()
cluster.close()

In [3]:
%%time
nwm.request_timeseries()
nwm.set_output()
nwm.data

<xarray.Dataset>
Dimensions:     (feature_id: 1, time: 768)
Coordinates:
  * feature_id  (feature_id) int32 6177558
    latitude    (feature_id) float32 41.31
    longitude   (feature_id) float32 -72.53
  * time        (time) datetime64[ns] 2010-01-01 ... 2010-02-01T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 0.51 0.51 0.52 0.55 ... 0.84 0.7 0.6
    velocity    (time, feature_id) float64 0.44 0.45 0.45 0.46 ... 0.53 0.5 0.47
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [4]:
%%time
results = nwm.output.to_dict()
results

Wall time: 0 ns


{'source': 'nwm',
 'dataset': 'streamflow',
 'metadata': {'retrieval_timestamp': '2021-05-12T10:19:47.053237',
  'source_url': 's3://noaa-nwm-retro-v2-zarr-pds',
  'Conventions': 'CF-1.6',
  'cdm_datatype': 'Station',
  'code_version': 'v5.1.0-alpha11',
  'dev': 'dev_ prefix indicates development/internal meta data',
  'dev_NOAH_TIMESTEP': '3600',
  'dev_OVRTSWCRT': '1',
  'dev_channelBucket_only': '0',
  'dev_channel_only': '0',
  'featureType': 'timeSeries',
  'model_configuration': 'retrospective',
  'model_initialization_time': '2018-10-01_00:00:00',
  'model_output_type': 'channel_rt',
  'model_output_valid_time': '2018-12-28_00:00:00',
  'model_total_valid_times': '2208',
  'proj4': '+proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0 +lat_2=60.0 +lat_0=40.0 +lon_0=-97.0 +x_0=0 +y_0=0 +k_0=1.0 +nadgrids=@',
  'station_dimension': 'feature_id',
  'stream_order_output': '1',
  'column_1_units': 'm3 s-1',
  'column_2_units': 'ms-1',
  'column_1': 'streamflow-6177558',
  'column